In [1]:
print()

In [2]:
# LOAD PACKAGES
# -------------
# import Pkg; Pkg.add("CSV"); Pkg.add("GLPK"); Pkg.add("JuMP")
using JuMP  # Need to say it whenever we use JuMP
using CSV
using Cbc

In [18]:
# MODEL CONSTRUCTION
# ------------------
capstone = Model(with_optimizer(Cbc.Optimizer, tm_lim=20*1000)) 
# We also set the time limit, in milliseconds

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

In [19]:
# INPUT DATA
# ----------

# Data on student preferences
unit_costs = CSV.read("../preprocessing/unit_costs.csv",header=false)
print(size(unit_costs))

# Data on Pillars
capacities = CSV.read("../preprocessing/capacities.csv",header=false)
print(size(capacities))

# Data on Pillars
supplies = CSV.read("../preprocessing/supplies.csv",header=false)
print(size(supplies))

length = size(unit_costs)[1]
iterlen = 1:length

(6, 6)(6, 6)(6, 1)

1:6

In [20]:
# VARIABLES
# ---------

# the flow
@variable(capstone, flow[i=iterlen,j=iterlen] >= 0, Int)  

# binary variable taking value 1 if there is a flow
@variable(capstone, flowing[i=iterlen,j=iterlen], Bin)

print()

In [21]:
# CONSTRAINTS
# -----------

# flow can be nonzero only if flowing is 1
@constraint(capstone, smaller_1[i=iterlen, j=iterlen], 
    flow[i,j] <= flowing[i,j]*9999999)

# flowing can be nonzero only if capacity is nonzero
@constraint(capstone, smaller_2[i=iterlen, j=iterlen],
    flowing[i,j] <= capacities[i,j])

# Every node can only have one output
@constraint(capstone, output[i=1:(length-1)],
    sum(flowing[i,j] for j = iterlen) == 1)

# Number of cstcs
@constraint(capstone, num_cstcs, 
    sum(flowing[j,length] for j = iterlen) <= 2)

# Every node needs to exhaust its supply
@constraint(capstone, supplied[i=iterlen],
    sum(flow[i,j] for j = iterlen) - sum(flow[j,i] for j = iterlen) == supplies[i,1])

print()

In [22]:
# OBJECTIVE
# ---------

# Maximize the total utility
@objective(capstone, Min, 
    sum(unit_costs[i,j]*flow[i,j] for i = iterlen, j = iterlen))

# print(capstone) # Shows the model constructed in a human-readable form
print()

In [23]:
# SOLVE IT
# --------

status = optimize!(capstone)
print()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Oct  7 2019 

command line - Cbc_C_Interface -tm_lim 20000 -solve -quit (default strategy 1)
No match for tm_lim - ? for list of commands
No match for 20000 - ? for list of commands
Continuous objective value is 118 - 0.00 seconds
Cgl0002I 23 variables fixed
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 6 tightened bounds, 6 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 4 tightened bounds, 4 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 5 tightened bounds, 5 strengthened rows, 0 substitutions
Cgl00

In [24]:
# DISPLAY THE RESULTS
# -------------------

println("Objective value: ", JuMP.objective_value(capstone))
println("flow = ", JuMP.value.(flow))
println("flowing = ", JuMP.value.(flowing))

Objective value: 187.0
flow = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, 1:6
And data, a 6×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  6.0
 5.0  0.0  0.0  0.0  0.0  0.0
 0.0  3.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  4.0  0.0
 0.0  0.0  0.0  0.0  0.0  9.0
 0.0  0.0  0.0  0.0  0.0  0.0
flowing = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:6
    Dimension 2, 1:6
And data, a 6×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  1.0
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0
